# 多层感知机 --- 从0开始 
前面我们介绍了包括线性回归和多类逻辑回归的数个模型，他们的共同点是全是只含有一个
输入层，一个输出层。这一节我们将介绍多层神经网络，就是包含至少一个隐含层的网络。 

# 数据获取 
使用FashionMNIST数据集 

In [1]:
import sys
import mxnet as mx

sys.path.append('..') 
from d2lzh_my import utils
batch_size=256
train_data,test_data = utils.load_data_fashion_mnist(batch_size) 

In [2]:
from mxnet import ndarray as nd 
num_inputs=28*28 
num_outputs=10 

num_hidden=256
weight_scale=0.01 

W1=nd.random_normal(shape=(num_inputs,num_hidden),scale=weight_scale)
b1=nd.zeros(num_hidden) 

W2=nd.random_normal(shape=(num_hidden,num_outputs),scale=weight_scale) 
b2=nd.zeros(num_outputs) 

params =[W1,b1,W2,b2]
for param in params:
    param.attach_grad()

# 激活函数 
如果我们就用线性操作符来构造多层神经网络，那么整个模型仍然%gui是一个线性函数。
这是因为 
   
这里。为了让我们的模型可以拟合非线性函数，我们需要在层之间插入非线性的激活函数。
这里我们使用Relu 
            Relu(X)=max(X,0) 

In [3]:
def relu(x):
    return nd.maximum(x,0) 

# 定义模型
我们的模型就是将层（全连接）和激活函数（Relu)串起来 


In [4]:
def net(X):
    X=X.reshape((-1,num_inputs)) 
    h1=relu(nd.dot(X,W1)+b1) 
    output=nd.dot(h1,W2)+b2
    return output 

# Softmax 和交叉熵损失函数 
在多类Logistic 回归里我们提到分开实现Softmax和交叉熵损失函数可能导致数值不稳定。这里
我们使用Gluon提供的函数 。 

In [5]:
from mxnet import gluon 
softmax_cross_entropy=gluon.loss.SoftmaxCrossEntropyLoss() 

# 训练 
训练和之前一样 

In [14]:
from mxnet import autograd as autograd 
earning_rate=.5 
for epoch in range(5):
    train_loss=0 
    train_acc=0 
    for data,label in train_data:
        with autograd.record():
            output=net(data)
            loss=softmax_cross_entropy(output,label)
        loss.backward() 
        utils.SGD(params,earning_rate/batch_size)  
        train_loss+=nd.mean(loss).asscalar()
        train_acc+=utils.accuracy(output,label.astype("float32")).asscalar() 
    test_acc = utils.evaluate_accuracy(test_data,net )  
    print("Epoch %d .loss : %f ,Train acc %f , Test acc %f "% (epoch,
                                                               train_loss/len(train_data),
                                                               train_acc/len(train_data),
                                                               test_acc))
   

Epoch 0 .loss : 0.302240 ,Train acc 0.887749 , Test acc 0.882200 
Epoch 1 .loss : 0.295959 ,Train acc 0.889533 , Test acc 0.886100 
Epoch 2 .loss : 0.287953 ,Train acc 0.893057 , Test acc 0.885500 
Epoch 3 .loss : 0.283267 ,Train acc 0.894287 , Test acc 0.887200 
Epoch 4 .loss : 0.279283 ,Train acc 0.897756 , Test acc 0.884600 
